In [1]:
import numpy as np
from sklearn import datasets
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import to_categorical
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import time
import copy
min_T = 0.00001

def annealing(func,x,T):
    '''
    func - h function
    x - argument 
    T - temperature
    return p(x)
    '''
    return np.power(np.e,(-func(x)/T))

def sa_algorithm(func, g_x, n, max_iter=300, an_rate=0.99):
    '''
    func - function to optimize
    g_x - proposed distribution
    max_iter - maximum number of iterations
    an_rate - annealing rate
    '''
    x_t = np.random.random(n)-1 # uniform [-0.5;0.5]
    global_sol = copy.deepcopy(x_t)
    T = 2000
    for iter in range(max_iter):
        x = g_x(x_t)
        alpha = annealing(func,x,T)/annealing(func,x_t,T)
        if (np.random.random()<=alpha): 
            x_t = copy.deepcopy(x)
        if func(global_sol)>=func(x_t): # update global min
            global_sol =copy.deepcopy(x_t)
        T = T*an_rate
        if T < min_T:
            break
    
    return global_sol

## Task 1


In [2]:
import random
random.seed(0)
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [3]:
def test_accuracy(model):
    return accuracy_score(y_test,np.argmax(model.predict(X_test),axis=1))

def train_accuracy(model):
    return accuracy_score(y_train,np.argmax(model.predict(X_train),axis=1))

In [13]:
model = Sequential()
model.add(Dense(4, input_dim=4, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
st = time.time()
model.fit(X_train,to_categorical(y_train),epochs=300,verbose = False)
print("Time s: ",  time.time()-st)
print("Train accuracy using backpropagation: ",train_accuracy(model))
print("Test accuracy using backpropagation: ",test_accuracy(model))

Time s:  3.7916128635406494
Train accuracy using backpropagation:  0.9583333333333334
Test accuracy using backpropagation:  0.9666666666666667


In [5]:
def get_g_x(n):
    return lambda x: np.random.normal(x,1,n)

def set_params(x,model):
    '''
    x - params
    nn - neural network model
    '''
    
    new_weights=[]
    current =0 # current weights
    for weight in model.get_weights():
        nps = np.prod(weight.shape) #number of parameters
        new_weights.append(np.reshape(x[current:current+nps],weight.shape))
        current+=nps
        
    model.set_weights(new_weights)
    
    return model
    
def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/N
    return ce

def loss(x):

    '''
    This is the function that will be optimized.
    x - parameters
    '''
    set_params(x,model)
    y = model.predict(X_train)
    loss_value = cross_entropy(to_categorical(y_train),y)
   
    
    return loss_value

In [6]:
st = time.time()
best_params = sa_algorithm(loss,get_g_x(model.count_params()),model.count_params(),an_rate=0.999)
set_params(best_params,model)
print("Time s:", time.time()-st)
print("Annealing rate 0.999, train accuracy:",train_accuracy(model))
print("Annealing rate 0.999, test accuracy:",test_accuracy(model))

Time s: 45.54397130012512
Annealing rate 0.999, train accuracy: 0.6583333333333333
Annealing rate 0.999, test accuracy: 0.7


In [7]:
st = time.time()
best_params = sa_algorithm(loss,get_g_x(model.count_params()),model.count_params(),an_rate=0.99)
set_params(best_params,model)
print("Time s:", time.time()-st)
print("Annealing rate 0.99, train accuracy:",train_accuracy(model))
print("Annealing rate 0.99, test accuracy:",test_accuracy(model))


Time s: 46.19804787635803
Annealing rate 0.99, train accuracy: 0.6666666666666666
Annealing rate 0.99, test accuracy: 0.6333333333333333


In [8]:
best_params = sa_algorithm(loss,get_g_x(model.count_params()),model.count_params(),an_rate=0.9)
print("Annealing rate 0.90, train accuracy:",train_accuracy(model))
print("Annealing rate 0.90, test accuracy:",test_accuracy(model))

C:\Users\arsee\Anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


Annealing rate 0.90, train accuracy: 0.675
Annealing rate 0.90, test accuracy: 0.6333333333333333


In [10]:
best_params = sa_algorithm(loss,get_g_x(model.count_params()),model.count_params(),an_rate=0.8)
print("Annealing rate 0.8, train accuracy:",train_accuracy(model))
print("Annealing rate 0.8, test accuracy:",test_accuracy(model))

C:\Users\arsee\Anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


Annealing rate 0.8, train accuracy: 0.5666666666666667
Annealing rate 0.8, test accuracy: 0.6


In [9]:
best_params = sa_algorithm(loss,get_g_x(model.count_params()),model.count_params(),an_rate=0.5)
print("Annealing rate 0.5, train accuracy:",train_accuracy(model))
print("Annealing rate 0.5, test accuracy:",test_accuracy(model))

C:\Users\arsee\Anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


Annealing rate 0.5, train accuracy: 0.3333333333333333
Annealing rate 0.5, test accuracy: 0.3333333333333333
